# Adjust contam CVF files

In [2]:
import numpy  as np
import pandas as pd

### 1. Set directory and filename

In [3]:
d = '/mnt/data/lima/iaq/contam_modeling/modeling_test_homes/wth_ctm_cvf/'
fn = 'Atmo7S_SmartHomesData.cvf'

### 2. Insert times of value changes

In [4]:
data = np.array(
[["2020-09-20 23:00:00","2020-09-22 16:30:00",0.000,0.000,1.000],
 ["2020-09-22 16:30:00","2020-09-23 17:00:00",1.000,0.005,1.000],
 ["2020-09-23 17:00:00","2020-09-29 16:30:00",1.000,0.005,0.000],
 ["2020-09-29 16:30:00","2020-09-30 16:30:00",1.000,0.005,1.000],
 ["2020-09-30 16:30:00","2020-10-03 16:30:00",0.000,0.000,1.000]]
               )

### 3. Convert data to a Pandas dataframe for processing

In [5]:
df = pd.DataFrame({'btime':  pd.to_datetime(data[:,0]),
                   'etime':  pd.to_datetime(data[:,1]),
                   'value1': data[:,2],
                   'value2': data[:,3],
                   'value3': data[:,4]})

df

,btime,etime,value1,value2,value3
0,2020-09-20 23:00:00,2020-09-22 16:30:00,0.0,0.0,1.0
1,2020-09-22 16:30:00,2020-09-23 17:00:00,1.0,0.005,1.0
2,2020-09-23 17:00:00,2020-09-29 16:30:00,1.0,0.005,0.0
3,2020-09-29 16:30:00,2020-09-30 16:30:00,1.0,0.005,1.0
4,2020-09-30 16:30:00,2020-10-03 16:30:00,0.0,0.0,1.0


### 4. Open a new file for adjusted data and process data

In [15]:
# Create new file for adjusted data
fa = open(d + fn + '.adjusted', 'wt')

# Open original file and determine where end-of-file (eof) is
fo = open(d + fn, 'rt')
fo.seek(0,2); eof = fo.tell(); fo.seek(0);

In [16]:
cnt = 0
btime  = df.iloc[cnt].btime.strftime('%m/%d\t%H:%M:%S')
etime  = df.iloc[cnt].etime.strftime('%m/%d\t%H:%M:%S')
value1 = df.iloc[cnt].value1
value2 = df.iloc[cnt].value2
value3 = df.iloc[cnt].value3

# Skip header lines
line = fo.readline()
while line.find(btime)<0:
    fa.write(line);
    line = fo.readline()

# Now process data lines and add values
while fo.tell() < eof:
    if line.find(etime)<0:
        fa.write(line[:-1] + '\t' + value1 + '\t' + value2 + '\t' + value3 + '\n');
        line = fo.readline()
    else:
        try:
            cnt    = cnt + 1
            btime  = df.iloc[cnt].btime.strftime('%m/%d\t%H:%M:%S')
            etime  = df.iloc[cnt].etime.strftime('%m/%d\t%H:%M:%S')
            value1 = df.iloc[cnt].value1
            value2 = df.iloc[cnt].value2
            value3 = df.iloc[cnt].value3
            # print(df.iloc[cnt])
        except:
            break

# When EOF is reached, write the last line
fa.write(line[:-1] + '\t' + value1 + '\t' + value2 + '\t' + value3 + '\n');

# Close both files
fa.close()
fo.close()